In [1]:
from skmultilearn.dataset import load_dataset

In [2]:
X_train, y_train, _, _ = load_dataset("bibtex","train")
X_test, y_test, _, _ = load_dataset("bibtex","test")

bibtex:train - exists, not redownloading
bibtex:test - exists, not redownloading


In [3]:
X_train, y_train

(<4880x1836 sparse matrix of type '<class 'numpy.float64'>'
 	with 334250 stored elements in List of Lists format>,
 <4880x159 sparse matrix of type '<class 'numpy.int64'>'
 	with 11616 stored elements in List of Lists format>)

In [7]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.svm import SVC

In [8]:
clf = BinaryRelevance(
    classifier=SVC(),
    require_dense=[False, True]
)

In [9]:
clf.classifiers

AttributeError: 'BinaryRelevance' object has no attribute 'classifiers'

In [10]:
clf.fit(X_train, y_train)

BinaryRelevance(classifier=SVC(), require_dense=[False, True])

In [11]:
prediction = clf.predict(X_test)

In [12]:
import sklearn.metrics as metrics

In [13]:
metrics.hamming_loss(y_test, prediction)

0.01314377883641547

In [14]:
metrics.accuracy_score(y_test, prediction)

0.11888667992047713

In [20]:
from skmultilearn.cluster.graphtool import StochasticBlockModel
from sklearn.ensemble import RandomForestClassifier
from skmultilearn.problem_transform import LabelPowerset
from skmultilearn.cluster.base import LabelCooccurrenceGraphBuilder
from skmultilearn.cluster.igraph import IGraphLabelGraphClusterer
from skmultilearn.ensemble import LabelSpacePartitioningClassifier

# construct base forest classifier
base_classifier = RandomForestClassifier(n_estimators=1000)

# construct a graph builder that will include
# label relations weighted by how many times they
# co-occurred in the data, without self-edges
graph_builder = LabelCooccurrenceGraphBuilder(
    weighted = True,
    include_self_edges = False
)

# select parameters for the model, we fit a flat,
# non-degree correlated, partitioning model
# which will use fit the normal distribution as the weights model
model = StochasticBlockModel(
    nested=False,
    use_degree_correlation=True,
    allow_overlap=False,
    weight_model='real-normal'
)

# setup problem transformation approach with sparse matrices for random forest
problem_transform_classifier = LabelPowerset(classifier=base_classifier,
    require_dense=[False, False])

# setup the clusterer to use, we selected the fast greedy modularity-maximization approach
clusterer = GraphToolLabelGraphClusterer(graph_builder=graph_builder, model=model)

# setup the ensemble metaclassifier
classifier = LabelSpacePartitioningClassifier(problem_transform_classifier, clusterer)

# train
classifier.fit(X_train, y_train)

# predict
predictions = classifier.predict(X_test)

ModuleNotFoundError: No module named 'graph_tool'